In [3]:
!pip install selenium


   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.4 MB 1.2 MB/s eta 0:00:08
   -- ------------------------------------- 0.5/9.4 MB 1.2 MB/s eta 0:00:08
   --- ------------------------------------ 0.8/9.4 MB 1.2 MB/s eta 0:00:08
   ---- ----------------------------------- 1.0/9.4 MB 1.2 MB/s eta 0:00:07
   ------ --------------------------------- 1.6/9.4 MB 1.2 MB/s eta 0:00:07
   ------ --------------------------------- 1.6/9.4 MB 1.2 MB/s eta 0:00:07
   -------- ------------------------------- 2.1/9.4 MB 1.2 MB/s eta 0:00:06
   -------- ------------------------------- 2.1/9.4 MB 1.2 MB/s eta 0:00:06
   ---------- ----------------------------- 2.4/9.4 MB 1.2 MB/s eta 0:00:06
   ------------ --------------------------- 2.9/9.4 MB 1.2 MB/s eta 0:00:06
   ------------ --------------------------- 2.9/9.4 MB 1.2 MB/s eta 0:00:06
   ------------- ---------


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**СОБИРАЊЕ НА ПОДАТОЦИ И КРЕИРАЊЕ НА ДАТАСЕТИ ОД ПРЕБАРАНИ САЈТОВИ КОИ ПРОДАВААТ ЛАПТОПИ ВО МАКЕДОНИЈА**


**НЕПТУН**

In [7]:
!pip install pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.5 MB 615.2 kB/s eta 0:00:18
   - -------------------------------------- 0.5/11.5 MB 615.2 kB/s eta 0:00:18
   -- ------------------------------------- 0.8/11.5 MB 685.4 kB/s eta 0:00:16
   --- ------------------------------------ 1.0/11.5 MB 731.1 kB/s eta 0:00:15
   --- ------------------------------------ 1.0/11.5 MB 731.1 kB/s eta 0:00:15
   ---- ----------------------------------- 1.3/11.5 MB 715.5 kB/s eta 0:00:15
   ---- ----------------------------------- 1.3/11.5 MB 715.5 kB/s eta 0:00:15
   ----- ---------------------------------- 1.6/11.5 MB 711.3 kB/s eta 0:00:14
   ----- ---------


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [96]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import html
import re

# ---- Helpers ----
def normalize_price(val):
    if not val or "No price" in val:
        return None
    return float(val.replace("ден.", "").replace("MKD", "").replace(",", "").replace(".", "").strip())

def extract_storage(title):
    if not isinstance(title, str):
        return None
    title = title.upper().replace(" / ", " ").replace("+", " ")
    title = re.sub(r'([A-Z]+)(\d+GB)', r'\1 \2', title)
    title = re.sub(r'(\d+GB)(\d+(?:GB|TB))', r'\1 \2', title)
    matches = re.findall(r'(\d+(?:TB|GB))( SSD| HDD)?', title)
    results = set()
    for size, dtype in matches:
        num = int(re.findall(r'\d+', size)[0])
        if size.endswith("TB") or num >= 128:
            results.add(f"{size} {dtype.strip() if dtype else 'SSD'}")
    return " / ".join(results) if results else None

def extract_ram(title):
    if not isinstance(title, str):
        return None
    match = re.search(r'(?<!\d)(\d{1,2})\s?GB(?!\s?(SSD|HDD))', title, re.IGNORECASE)
    return match.group(1) + " GB" if match else None

def extract_brand(title):
    if not isinstance(title, str):
        return "Notebook"
    title = title.lower()
    known_brands = ["hp", "lenovo", "asus", "acer", "apple", "dell", "msi", "gigabyte", "huawei", "microsoft", "razer"]
    if "thinkbook" in title or "loq" in title:
        return "Lenovo"
    for brand in known_brands:
        if brand in title:
            return brand.capitalize()
    match = re.search(r'лаптоп\s+([A-Za-zА-Яа-я0-9]+)', title)
    if match:
        return match.group(1).capitalize()
    return title.split()[0].capitalize() if title else "Notebook"

def extract_cpu(title):
    if not isinstance(title, str):
        return "Unknown"

    title = title.upper().replace("®", "").replace("™", "").replace("-", " ")
    title = re.sub(r"\s+", " ", title)

    patterns = [
        r'INTEL CORE ULTRA \d+ \d{3,5}[A-Z]*',
        r'INTEL CORE I[3579] \d{3,5}[A-Z]*',
        r'INTEL CORE I[3579]',
        r'INTEL I[3579] \d{3,5}[A-Z]*',
        r'I[3579] \d{3,5}[A-Z]*',
        r'I[3579]-\d{3,5}[A-Z]*',
        r'INTEL \d',
        r'CORE I[3579] \d{3,5}[A-Z]*',
        r'CORE [3579]',
        r'INTEL [3579]',
        r'AMD RYZEN \d \d{3,5}[A-Z]*',
        r'RYZEN \d \d{3,5}[A-Z]*',
        r'RYZEN\d \d{3,5}[A-Z]*',
        r'RYZEN[3579] \d{3,5}[A-Z]*',
        r'RYZEN [3579]',
        r'RYZEN[3579]',
        r'R[3579] \d{3,5}[A-Z]*',
        r'R[3579]-\d{3,5}[A-Z]*',
        r'APPLE M[1234]',
        r'M[1234]',
        r'INTEL UHD',
        r'INTEL XE',
        r'VEGA \d+',
        r'ADRENO \d+',
        r'SNAPDRAGON\s+[A-Z]*\s?X?[A-Z]*[\w\-]+',
        r'CORE\s+ULTRA\s+[Uu]?\d[- ]?\d{3,5}[A-Z]*',
        r'ULTRA\s+\d\s+\d{3,5}[A-Z]*',
        r'U\d-\d{3,5}[A-Z]*',                 
        r'ULTRA\s+\d\s+\d{3,5}[A-Z]*',
        r'RYZEN\s+AI\s+\d\s+HX\s+\d{3}',      
        r'R\d-AI\s+\d{3}',                    
        r'R\d\s+AI\s+\d{3}',
        r'PENTIUM\s?\d{0,5}[A-Z]*',
        r'CELERON\s?\d{0,5}[A-Z]*'

    ]

    for pattern in patterns:
        match = re.search(pattern, title)
        if match:
            return match.group(0).strip()

    return "Unknown"


# ---- Setup ----
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
service = Service()
driver = webdriver.Chrome(service=service, options=options)
results = []

# -------- ANHOCH --------
for page in range(1, 6):
    driver.get(f"https://www.anhoch.com/categories/site-laptopi/products?brand=&attribute=&toPrice=324980&inStockOnly=2&sort=latest&perPage=20&page={page}")
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    products = soup.find_all("div", class_="product-card")
    for product in products:
        title = product.get("title")
        price_tag = product.find("div", class_="product-price")
        price = price_tag.get_text(strip=True).split("ден")[0].strip() + " ден." if price_tag else "No price"
        discount_tag = price_tag.find("span", class_="previous-price") if price_tag else None
        discount_price = discount_tag.get_text(strip=True) if discount_tag else None
        p = normalize_price(price)
        dp = normalize_price(discount_price)
        if p and dp and dp > p:
            price, discount_price = discount_price, price
        results.append({
            "Prodavnica": "Anhoch",
            "Ime": title,
            "Brend": extract_brand(title),
            "Procesor": extract_cpu(title),
            "RAM_Memorija": extract_ram(title),
            "Kapacitet": extract_storage(title),
            "Cena": price,
            "Klub/Smart/Happy cena": discount_price
        })

# -------- TEHNOMARKET --------
driver.get("https://www.tehnomarket.com.mk/category/4003/laptopi")
time.sleep(5)
soup = BeautifulSoup(driver.page_source, "html.parser")
products = soup.find_all("li", class_="span4")
for product in products:
    title_tag = product.find("div", class_="product-name")
    title = title_tag.text.strip() if title_tag else "No title"
    smart_price_tag = product.find("div", class_="smart-products")
    regular_price_tag = product.find("div", string=lambda t: t and "Редовна Цена" in t)
    price = smart_price_tag.find("span", class_="nm").text.strip() + " ден." if smart_price_tag else (
        regular_price_tag.find("span", class_="nm").text.strip() + " ден." if regular_price_tag else "No price")
    discount_price = regular_price_tag.find("span", class_="nm").text.strip() + " ден." if regular_price_tag else None
    p = normalize_price(price)
    dp = normalize_price(discount_price)
    if p and dp and dp > p:
        price, discount_price = discount_price, price
    results.append({
        "Prodavnica": "Tehnomarket",
        "Ime": title,
        "Brend": extract_brand(title),
        "Procesor": extract_cpu(title),
        "RAM_Memorija": extract_ram(title),
        "Kapacitet": extract_storage(title),
        "Cena": price,
        "Klub/Smart/Happy cena": discount_price
    })

# -------- GJIRAFA50 --------
for page in range(1, 6):
    driver.get(f"https://gjirafa50.mk/gaming-laptop?pagenumber={page}")
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    products = soup.find_all("div", class_="item-box")
    for product in products:
        title_tag = product.find("h3", class_="product-title")
        title = title_tag.text.strip() if title_tag else "No title"
        price_tag = product.find("span", class_="price")
        price = price_tag.text.strip() if price_tag else "No price"
        discount_tag = product.find("span", class_="old-price")
        discount_price = discount_tag.text.strip() if discount_tag else None
        p = normalize_price(price)
        dp = normalize_price(discount_price)
        if p and dp and dp > p:
            price, discount_price = discount_price, price
        results.append({
            "Prodavnica": "Gjirafa50",
            "Ime": html.unescape(title),
            "Brend": extract_brand(title),
            "Procesor": extract_cpu(title),
            "RAM_Memorija": extract_ram(title),
            "Kapacitet": extract_storage(title),
            "Cena": price,
            "Klub/Smart/Happy cena": discount_price
        })

# -------- NEPTUN --------
for page in range(1, 6):
    driver.get(f"https://www.neptun.mk/prenosni_kompjuteri.nspx?page={page}")
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    products = soup.find_all("div", class_="white-box")
    if not products:
        break
    for product in products:
        title_tag = product.find("h2", class_="product-list-item__content--title")
        price_tag = product.find("span", class_="product-price__amount--value")
        discount_tag = product.find("span", class_="product-price__amount--discount")
        title = title_tag.get_text(strip=True) if title_tag else "No title"
        price = price_tag.get_text(strip=True) if price_tag else "No price"
        discount_price = discount_tag.get_text(strip=True) if discount_tag else None
        p = normalize_price(price)
        dp = normalize_price(discount_price)
        if p and dp and dp > p:
            price, discount_price = discount_price, price
        results.append({
            "Prodavnica": "Neptun",
            "Ime": title,
            "Brend": extract_brand(title),
            "Procesor": extract_cpu(title),
            "RAM_Memorija": extract_ram(title),
            "Kapacitet": extract_storage(title),
            "Cena": price,
            "Klub/Smart/Happy cena": discount_price
        })

# ---- Export ----
driver.quit()
df = pd.DataFrame(results)
df["Klub/Smart/Happy cena"] = df["Klub/Smart/Happy cena"].where(df["Klub/Smart/Happy cena"].notnull(), pd.NA)
df.to_csv("laptops_merged.csv", index=False, encoding='utf-8-sig')
print(df.head())
print(f"\n✅ Total products scraped: {len(df)}")
print("📄 Saved as 'laptops_merged.csv'")


  Prodavnica                                                Ime   Brend  \
0     Anhoch  Notebook HP 15 i5-1334U 16GB/512GB/15.6" FHD I...      Hp   
1     Anhoch  Notebook HP Pavilion 15 Ryzen7 7730U 16GB/512G...      Hp   
2     Anhoch  Notebook Lenovo IdeaPad Slim 3 Ryzen3 7320U/8G...  Lenovo   
3     Anhoch  Notebook Lenovo IdeaPad 1 Ryzen5 5500U 8GB/512...  Lenovo   
4     Anhoch  Notebook Lenovo IdeaPad Slim 3 Core 5 120U/16G...  Lenovo   

       Procesor RAM_Memorija  Kapacitet            Cena Klub/Smart/Happy cena  
0      I5 1334U        16 GB  512GB SSD  25.980,00 ден.                  <NA>  
1  RYZEN7 7730U        16 GB  512GB SSD  27.980,00 ден.        26.980,00 ден.  
2  RYZEN3 7320U         8 GB  256GB SSD  16.980,00 ден.                  <NA>  
3  RYZEN5 5500U         8 GB  512GB SSD  18.980,00 ден.        17.980,00 ден.  
4        CORE 5        16 GB  512GB SSD  30.980,00 ден.                  <NA>  

✅ Total products scraped: 402
📄 Saved as 'laptops_merged.csv'


**DATA PREPROCESSING**

In [101]:
import pandas as pd

# Load the dataset
df1 = pd.read_csv("laptops_merged.csv")

# Quick inspection
print(df1.columns)
print(df1.head())


Index(['Prodavnica', 'Ime', 'Brend', 'Procesor', 'RAM_Memorija', 'Kapacitet',
       'Cena', 'Klub/Smart/Happy cena'],
      dtype='object')
  Prodavnica                                                Ime   Brend  \
0     Anhoch  Notebook HP 15 i5-1334U 16GB/512GB/15.6" FHD I...      Hp   
1     Anhoch  Notebook HP Pavilion 15 Ryzen7 7730U 16GB/512G...      Hp   
2     Anhoch  Notebook Lenovo IdeaPad Slim 3 Ryzen3 7320U/8G...  Lenovo   
3     Anhoch  Notebook Lenovo IdeaPad 1 Ryzen5 5500U 8GB/512...  Lenovo   
4     Anhoch  Notebook Lenovo IdeaPad Slim 3 Core 5 120U/16G...  Lenovo   

       Procesor RAM_Memorija  Kapacitet            Cena Klub/Smart/Happy cena  
0      I5 1334U        16 GB  512GB SSD  25.980,00 ден.                   NaN  
1  RYZEN7 7730U        16 GB  512GB SSD  27.980,00 ден.        26.980,00 ден.  
2  RYZEN3 7320U         8 GB  256GB SSD  16.980,00 ден.                   NaN  
3  RYZEN5 5500U         8 GB  512GB SSD  18.980,00 ден.        17.980,00 ден.  
4       

In [102]:
df1

,Prodavnica,Ime,Brend,Procesor,RAM_Memorija,Kapacitet,Cena,Klub/Smart/Happy cena
0,Anhoch,"Notebook HP 15 i5-1334U 16GB/512GB/15.6"" FHD I...",Hp,I5 1334U,16 GB,512GB SSD,"25.980,00 ден.",NaN
1,Anhoch,Notebook HP Pavilion 15 Ryzen7 7730U 16GB/512G...,Hp,RYZEN7 7730U,16 GB,512GB SSD,"27.980,00 ден.","26.980,00 ден."
2,Anhoch,Notebook Lenovo IdeaPad Slim 3 Ryzen3 7320U/8G...,Lenovo,RYZEN3 7320U,8 GB,256GB SSD,"16.980,00 ден.",NaN
3,Anhoch,Notebook Lenovo IdeaPad 1 Ryzen5 5500U 8GB/512...,Lenovo,RYZEN5 5500U,8 GB,512GB SSD,"18.980,00 ден.","17.980,00 ден."
4,Anhoch,Notebook Lenovo IdeaPad Slim 3 Core 5 120U/16G...,Lenovo,CORE 5,16 GB,512GB SSD,"30.980,00 ден.",NaN
...,...,...,...,...,...,...,...,...
397,Neptun,No title,No,Unknown,NaN,NaN,No price,NaN
398,Neptun,No title,No,Unknown,NaN,NaN,No price,NaN
399,Neptun,No title,No,Unknown,NaN,NaN,No price,NaN
400,Neptun,No title,No,Unknown,NaN,NaN,No price,NaN


In [103]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Prodavnica             402 non-null    object
 1   Ime                    402 non-null    object
 2   Brend                  402 non-null    object
 3   Procesor               402 non-null    object
 4   RAM_Memorija           351 non-null    object
 5   Kapacitet              337 non-null    object
 6   Cena                   402 non-null    object
 7   Klub/Smart/Happy cena  132 non-null    object
dtypes: object(8)
memory usage: 25.3+ KB


In [104]:
df1.isnull().sum()

Prodavnica                 0
Ime                        0
Brend                      0
Procesor                   0
RAM_Memorija              51
Kapacitet                 65
Cena                       0
Klub/Smart/Happy cena    270
dtype: int64

**Чистење на цените**

In [105]:
def clean_price(value):
    if pd.isna(value):
        return None
    value = str(value).replace("ден.", "").replace("MKD", "").replace(",", "").replace(".", "").strip()
    try:
        return float(value)
    except ValueError:
        return None

df1["Cena"] = df1["Cena"].apply(clean_price)
df1["Klub/Smart/Happy cena"] = df1["Klub/Smart/Happy cena"].apply(clean_price)


In [106]:
df1

,Prodavnica,Ime,Brend,Procesor,RAM_Memorija,Kapacitet,Cena,Klub/Smart/Happy cena
0,Anhoch,"Notebook HP 15 i5-1334U 16GB/512GB/15.6"" FHD I...",Hp,I5 1334U,16 GB,512GB SSD,2598000.0,NaN
1,Anhoch,Notebook HP Pavilion 15 Ryzen7 7730U 16GB/512G...,Hp,RYZEN7 7730U,16 GB,512GB SSD,2798000.0,2698000.0
2,Anhoch,Notebook Lenovo IdeaPad Slim 3 Ryzen3 7320U/8G...,Lenovo,RYZEN3 7320U,8 GB,256GB SSD,1698000.0,NaN
3,Anhoch,Notebook Lenovo IdeaPad 1 Ryzen5 5500U 8GB/512...,Lenovo,RYZEN5 5500U,8 GB,512GB SSD,1898000.0,1798000.0
4,Anhoch,Notebook Lenovo IdeaPad Slim 3 Core 5 120U/16G...,Lenovo,CORE 5,16 GB,512GB SSD,3098000.0,NaN
...,...,...,...,...,...,...,...,...
397,Neptun,No title,No,Unknown,NaN,NaN,NaN,NaN
398,Neptun,No title,No,Unknown,NaN,NaN,NaN,NaN
399,Neptun,No title,No,Unknown,NaN,NaN,NaN,NaN
400,Neptun,No title,No,Unknown,NaN,NaN,NaN,NaN


**Чистење/Стандардизирање на колоната за РАМ меморија**

In [107]:
import re

def standardize_ram(ram):
    if pd.isna(ram):
        return None
    match = re.search(r'(\d{1,2})', str(ram))
    return int(match.group(1)) if match else None

df1["RAM_Memorija"] = df1["RAM_Memorija"].apply(standardize_ram)


**Станданрдизирање на капацитетот**

In [110]:
def standardize_storage(val):
    if pd.isna(val):
        return None
    val = val.upper().replace("SSD", " SSD").replace("HDD", " HDD")
    parts = [p.strip() for p in val.split("/") if p.strip()]
    return " / ".join(sorted(set(parts)))

df1["Kapacitet"] = df1["Kapacitet"].apply(standardize_storage)

In [111]:
df1

,Prodavnica,Ime,Brend,Procesor,RAM_Memorija,Kapacitet,Cena,Klub/Smart/Happy cena
0,Anhoch,"Notebook HP 15 i5-1334U 16GB/512GB/15.6"" FHD I...",Hp,I5 1334U,16.0,512GB SSD,2598000.0,NaN
1,Anhoch,Notebook HP Pavilion 15 Ryzen7 7730U 16GB/512G...,Hp,RYZEN7 7730U,16.0,512GB SSD,2798000.0,2698000.0
2,Anhoch,Notebook Lenovo IdeaPad Slim 3 Ryzen3 7320U/8G...,Lenovo,RYZEN3 7320U,8.0,256GB SSD,1698000.0,NaN
3,Anhoch,Notebook Lenovo IdeaPad 1 Ryzen5 5500U 8GB/512...,Lenovo,RYZEN5 5500U,8.0,512GB SSD,1898000.0,1798000.0
4,Anhoch,Notebook Lenovo IdeaPad Slim 3 Core 5 120U/16G...,Lenovo,CORE 5,16.0,512GB SSD,3098000.0,NaN
...,...,...,...,...,...,...,...,...
397,Neptun,No title,No,Unknown,NaN,None,NaN,NaN
398,Neptun,No title,No,Unknown,NaN,None,NaN,NaN
399,Neptun,No title,No,Unknown,NaN,None,NaN,NaN
400,Neptun,No title,No,Unknown,NaN,None,NaN,NaN


**Претворање на капацитетот во вкупно GB меморија**

In [134]:
import numpy as np

def extract_total_storage_gb(val):
    if not isinstance(val, str):
        return None

    total = 0
    parts = val.upper().split("/")
    for part in parts:
        match = re.search(r'(\d+(?:\.\d+)?)(TB|GB)', part)
        if match:
            size, unit = match.groups()
            gb = float(size) * (1024 if unit == "TB" else 1)
            total += gb
    return int(total) if total > 0 else None

df1["Kapacitet"] = df1["Kapacitet"].apply(extract_total_storage_gb)



**Стандардизирање на имињата на процесорите, почисто и порубстно**

In [129]:
import re

def clean_procesor(cpu):
    if pd.isna(cpu) or cpu.strip().lower() == "unknown":
        return None
    cpu = cpu.strip().upper()
    
    # Standardize spacing
    cpu = re.sub(r'\s+', ' ', cpu)
    
    # Normalize common patterns
    cpu = cpu.replace("INTEL CORE ", "I").replace("CORE ", "I")
    cpu = cpu.replace("AMD RYZEN", "RYZEN")
    cpu = cpu.replace("RYZEN R", "RYZEN ")
    cpu = cpu.replace("M1", "M1").replace("M2", "M2").replace("M3", "M3")
    
    # Fix AI series spacing
    cpu = cpu.replace("HX ", "HX")

    return cpu.strip().title()

df1["Procesor"] = df1["Procesor"].apply(clean_procesor)


In [130]:
df1

,Prodavnica,Ime,Brend,Procesor,RAM_Memorija,Kapacitet,Cena,Klub/Smart/Happy cena,Vkupno_Kapacitet_GB,Kapacitet_GB
0,Anhoch,"HP 15 i5-1334U 16GB/512GB/15.6"" FHD IPS/Iris X...",Hp,I5 1334U,16.0,512.0,2598000.0,NaN,512.0,512.0
1,Anhoch,HP Pavilion 15 Ryzen7 7730U 16GB/512GB SSD/15....,Hp,Ryzen7 7730U,16.0,512.0,2798000.0,2698000.0,512.0,512.0
2,Anhoch,Lenovo IdeaPad Slim 3 Ryzen3 7320U/8GB/256GB S...,Lenovo,Ryzen3 7320U,8.0,256.0,1698000.0,NaN,256.0,256.0
3,Anhoch,Lenovo IdeaPad 1 Ryzen5 5500U 8GB/512GB SSD/15...,Lenovo,Ryzen5 5500U,8.0,512.0,1898000.0,1798000.0,512.0,512.0
4,Anhoch,Lenovo IdeaPad Slim 3 Core 5 120U/16GB/512GB S...,Lenovo,I5,16.0,512.0,3098000.0,NaN,512.0,512.0
...,...,...,...,...,...,...,...,...,...,...
397,Neptun,No title,No,None,NaN,NaN,NaN,NaN,NaN,NaN
398,Neptun,No title,No,None,NaN,NaN,NaN,NaN,NaN,NaN
399,Neptun,No title,No,None,NaN,NaN,NaN,NaN,NaN,NaN
400,Neptun,No title,No,None,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
df1.head(10)

,Prodavnica,Ime,Brend,Procesor,RAM_Memorija,Kapacitet,Cena,Klub/Smart/Happy cena,Vkupno_Kapacitet_GB,Kapacitet_GB
0,Anhoch,"HP 15 i5-1334U 16GB/512GB/15.6"" FHD IPS/Iris X...",Hp,I5 1334U,16.0,512.0,2598000.0,NaN,512.0,512.0
1,Anhoch,HP Pavilion 15 Ryzen7 7730U 16GB/512GB SSD/15....,Hp,Ryzen7 7730U,16.0,512.0,2798000.0,2698000.0,512.0,512.0
2,Anhoch,Lenovo IdeaPad Slim 3 Ryzen3 7320U/8GB/256GB S...,Lenovo,Ryzen3 7320U,8.0,256.0,1698000.0,NaN,256.0,256.0
3,Anhoch,Lenovo IdeaPad 1 Ryzen5 5500U 8GB/512GB SSD/15...,Lenovo,Ryzen5 5500U,8.0,512.0,1898000.0,1798000.0,512.0,512.0
4,Anhoch,Lenovo IdeaPad Slim 3 Core 5 120U/16GB/512GB S...,Lenovo,I5,16.0,512.0,3098000.0,NaN,512.0,512.0
5,Anhoch,"Lenovo IP3 i7-1255U 16GB/512GB/15.6"" FullHD/DO...",Lenovo,I7 1255U,16.0,512.0,2898000.0,NaN,512.0,512.0
6,Anhoch,Lenovo LOQ Gaming i5-12450HX/16GB/512GB/RTX405...,Lenovo,I5 12450Hx,16.0,512.0,4798000.0,4698000.0,512.0,512.0
7,Anhoch,"HP 15 i5-1334U 12GB/512GB/15.6"" FHD IPS/Iris X...",Hp,I5 1334U,12.0,512.0,2548000.0,2448000.0,512.0,512.0
8,Anhoch,Lenovo IdeaPad Slim 3 Ryzen7 5825U 16GB/512GB/...,Lenovo,Ryzen7 5825U,16.0,512.0,2498000.0,NaN,512.0,512.0
9,Anhoch,"HP Pavilion 15 i5-1335U 8GB/512GB SSD/15.6"" Fu...",Hp,I5 1335U,8.0,512.0,2898000.0,NaN,512.0,512.0


**Чистење на имињата, односно тргање на префикси пред имињата на моделите**

In [132]:
def clean_name(name):
    if not isinstance(name, str):
        return name
    # Remove common clutter words from the beginning
    return re.sub(r'^(Notebook|Лаптоп|Laptop)\s+', '', name, flags=re.IGNORECASE).strip()

df1["Ime"] = df1["Ime"].apply(clean_name)


In [133]:
df1.tail(150)

,Prodavnica,Ime,Brend,Procesor,RAM_Memorija,Kapacitet,Cena,Klub/Smart/Happy cena,Vkupno_Kapacitet_GB,Kapacitet_GB
252,Gjirafa50,"Lenovo LOQ Intel, 15.6"", Intel i5-12450HX, 16 ...",Lenovo,Intel I5 12450Hx,16.0,NaN,75339.0,68790.0,NaN,NaN
253,Gjirafa50,"Lenovo LOQ Intel, 15.6"", Intel i5-12450HX, 16 ...",Lenovo,Intel I5 12450Hx,16.0,NaN,58090.0,NaN,NaN,NaN
254,Gjirafa50,"Lenovo Yoga Pro, 16"", Intel 155H, 16 GB RAM, 5...",Lenovo,Intel 1,16.0,NaN,115159.0,111690.0,NaN,NaN
255,Gjirafa50,"MSI Thin 15, 15.6"", Intel i5-12450H, 16 GB RAM...",Msi,Intel I5 12450H,16.0,NaN,49790.0,48690.0,NaN,NaN
256,Gjirafa50,No title,No,None,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
397,Neptun,No title,No,None,NaN,NaN,NaN,NaN,NaN,NaN
398,Neptun,No title,No,None,NaN,NaN,NaN,NaN,NaN,NaN
399,Neptun,No title,No,None,NaN,NaN,NaN,NaN,NaN,NaN
400,Neptun,No title,No,None,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
df1.tail(50)

,Prodavnica,Ime,Brend,Procesor,RAM_Memorija,Kapacitet,Cena,Klub/Smart/Happy cena,Vkupno_Kapacitet_GB,Kapacitet_GB
352,Neptun,ACER Aspire 15 A15-41M-R5XN R7-8840HS/16GB DDR...,Acer,R7 8840Hs,16.0,None,40999.0,NaN,1024.0,1024.0
353,Neptun,"LENOVO IdeaPad Slim 5 14IMH9 14"" Ultra 5 125H/...",Lenovo,Ultra 5 125H,16.0,None,40999.0,NaN,512.0,512.0
354,Neptun,LENOVO IdeaPad Slim 3 15IRU9 Core 7 150U/16GB ...,Lenovo,I7,16.0,None,41999.0,NaN,512.0,512.0
355,Neptun,ACER Aspire Go 15 AG15-51P-77U9 15.3” FHD+/i7-...,Acer,I7 1355U,32.0,None,42999.0,NaN,1024.0,1024.0
356,Neptun,"ACER Swift Go 14 SFG14-42-R54G, R5-7640U/16GB/...",Acer,R5 7640U,16.0,None,46999.0,NaN,512.0,512.0
357,Neptun,"ACER Swift GO 14 SFG314-42-R1S7 14"" R5-7640U/1...",Acer,R5 7640U,16.0,None,48999.0,NaN,512.0,512.0
358,Neptun,"ACER Swift Go 14 SFG14-71-72JF 14"" i7-1355U/16...",Acer,I7 1355U,16.0,None,51999.0,NaN,512.0,512.0
359,Neptun,"ASUS Zenbook 14 OLED UX3405MA-QD438 14"" Ultra ...",Asus,Ultra 5 125H,16.0,None,53999.0,NaN,512.0,512.0
360,Neptun,"ASUS Vivobook S16 M5606KA-RI076 16"" 3K OLED 12...",Asus,R5 Ai 340,16.0,None,53999.0,NaN,512.0,512.0
361,Neptun,Apple MacBook Air 13.6 M2/8C CPU/8C GPU/16GB/2...,Apple,M2,16.0,None,56799.0,NaN,256.0,256.0
